In [1]:
import carla
import random

In [2]:
client = carla.Client('localhost', 2000)
world = client.get_world()

In [3]:
# Retrieve the spectator object
spectator = world.get_spectator()

# Get the location and rotation of the spectator through its transform
transform = spectator.get_transform()

location = transform.location
rotation = transform.rotation

# print(location, rotation)

In [4]:
# Set the spectator to a position
spectator.set_transform(carla.Transform(carla.Location(x=-62.010975, y=1.288139, z=17.589510),carla.Rotation(pitch=-48.045647, yaw=53.329460, roll=0.000444)))


Add Traffic

---------- RGB camera ----------

In [ ]:
camera_trans = carla.Transform(carla.Location(z=0.1))

camera_bp= world.get_blueprint_library().find('sensor.camera.rgb')

rgb_camera = world.spawn_actor(camera_bp, camera_trans, attach_to=spectator)

In [ ]:
rgb_camera.listen(lambda image: image.save_to_disk('rgb_output2/%06d.png' % image.frame))

In [ ]:
rgb_camera.stop()

---------- DVS camera ----------

In [5]:
camera_trans = carla.Transform(carla.Location(z=0.1))

dvs_camera_bp= world.get_blueprint_library().find('sensor.camera.dvs')

dvs_camera = world.spawn_actor(dvs_camera_bp, camera_trans, attach_to=spectator)

In [6]:
import csv
import numpy as np
import time
import cv2

In [7]:
def dvs_callback(data, dvs_output_path): # store in csv file
    print("length = ",len(data))

    dvs_events = np.frombuffer(data.raw_data, dtype=np.dtype([
        ('x', np.uint16), ('y',np.uint16), ('t',np.int64), ('pol', np.bool)]))
    with open(dvs_output_path, mode="a",  newline='') as file:
        writer = csv.writer(file)
        # dvs_events = np.frombuffer(data.raw_data, dtype=np.dtype([
        #     ('x', np.uint16), ('y',np.uint16), ('t',np.int64), ('pol', np.bool)]))
        for event in dvs_events:
            writer.writerow(event)
        file.close()
    

In [16]:
def dvs_callback_img(data): #store in image
    dvs_events = np.frombuffer(data.raw_data, dtype=np.dtype([
        ('x', np.uint16), ('y',np.uint16), ('t',np.int64), ('pol', np.bool)]))
    # data_dict['dvs_image'] = np.zeros((data.height, data.weight, 4), dtype=np.uint8)
    dvs_img = np.zeros((data.height, data.width, 3), dtype=np.uint8)
    dvs_img[dvs_events[:]['y'],dvs_events[:]['x'],dvs_events[:]['pol']*2] = 255

    # data_dict['dvs_image'][:,:,0:3] = dvs_img
    # print(dvs_img)
    # dvs_img.save_to_disk('output/%06d.png' % dvs_img.frame)
    cv2.imshow("dvs_imh", dvs_img)
    cv2.imwrite(f'output/{data.frame}.png', dvs_img)
    
    

In [8]:
dvs_output_path = "dvs_output.csv"
with open(dvs_output_path, mode="w",  newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['x', 'y', 't', 'pol'])
    file.close()

In [10]:
dvs_camera.stop()

: 

In [17]:
dvs_camera.listen(lambda DVSEventArray: dvs_callback_img(DVSEventArray))
# time.sleep(3)
# dvs_camera.stop()

In [9]:
dvs_camera.listen(lambda DVSEventArray: dvs_callback(DVSEventArray, dvs_output_path))
time.sleep(3)
dvs_camera.stop()

length =  7777
length =  8130
length =  9799
length =  9531
length =  10820
length =  10653
length =  9414
length =  10347
length =  10571
length =  10852
length =  10988
length =  10802
length =  11410
length =  11358
length =  11099
length =  11996
length =  11913


In [ ]:

dvs_output_path = "dvs_output.csv"

with open(dvs_output_path, mode="r",  newline='') as file:
    first = 0
    second = 0
    reader = csv.reader(file)
    for row in reader:
        second = int(row[2])
        if first > second:
            print("ERROR!!!")
            print(first, second)
        first = int(row[2])
    file.close()